<a href="https://colab.research.google.com/github/Hafizh-Al-Muzakar/Scrapping-film-merah-putih-one-for-all-twitter/blob/main/Scrapping_Film_Merah_Putih_Twiterr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install twscrape

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.2 MB/s eta 0:00:00


In [3]:
import asyncio
from twscrape import API
from twscrape.logger import set_log_level
import pandas as pd
import json

In [4]:
accounts = []
LIMIT_TWEET = 10000

In [6]:
with open("scrapping/accounts.json" , "r") as f:
  accounts = json.load(f)
if not accounts:
  print("No account found in account.json. please add account to scrape")
  exit(1)

In [18]:
async def scrape_tweets(api, start_date, end_date, save_to_file):
  tweets = []

  q = f"merah putih one for all since:{start_date} until:{end_date}"
  print(f"Scapping tweets : {q}")

  async for tweet in api.search(q, limit = LIMIT_TWEET):
    c =[
        tweet.id,
        tweet.date,
        tweet.url,
        tweet.user.username,
        tweet.rawContent,
        tweet.likeCount,
        tweet.replyCount,
        tweet.quoteCount,
        tweet.retweetCount
    ]
    tweets.append(c)

  df = pd.DataFrame(tweets,
                    columns = ["tweet_id","time_created","url","username","tweet","like_count","reply_count","quote_count","retweet_count"],
                    )
  file_path = "scrapping/results/" + save_to_file

  df.to_csv(file_path, index = False)
  print(f"Saved {len(df)} tweets to {file_path}")

  return len(df)

In [19]:
async def main():
    api = API()

    for account in accounts:
        cookies = f"auth_token={account['auth_token']}; ct0={account['ct0']}"
        await api.pool.add_account(
            account["username"],
            account["password"],
            account["email"],
            account["email_password"],
            cookies=cookies,
        )

    await api.pool.login_all()

    set_log_level("INFO")
    print("Starting scraping...")

    date_ranges = [
        ("2025-07-01", "2025-08-15", "tweets-merah-putih-one-for-all-2024-07.csv"),

    ]

    total_count = 0
    for start_date, end_date, file_name in date_ranges:
        count = await scrape_tweets(api, start_date, end_date, file_name)
        total_count += count


    print(f"\n🎉 Scraping completed. Total tweets scraped: {total_count}")

In [20]:
import nest_asyncio
nest_asyncio.apply()
await main()

2025-08-15 12:28:16.113 | WARNING  | twscrape.accounts_pool:add_account:88 - Account HMuzakar86311 already exists


Starting scraping...
Scapping tweets : merah putih one for all since:2025-07-01 until:2025-08-15


2025-08-15 12:28:48.836 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 12:43:16
2025-08-15 12:43:20.135 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:308 - Continuing with account HMuzakar86311 on queue SearchTimeline
2025-08-15 12:43:50.837 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 12:58:20
2025-08-15 12:58:22.158 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:308 - Continuing with account HMuzakar86311 on queue SearchTimeline
2025-08-15 12:58:53.247 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 13:13:22
2025-08-15 13:13:24.511 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:308 - Continuing with account HMuzakar86311 on queue SearchTimeline
2025-08-15 13:13:54.999 | INFO     | twscrape.accounts_poo

Saved 4632 tweets to scrapping/results/tweets-merah-putih-one-for-all-2024-06.csv

🎉 Scraping completed. Total tweets scraped: 4632
